In [4]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [5]:
data=pd.read_csv('../merged/len3_ndb_agg_blp_DropNever.csv',encoding="shift-jis",index_col=0)
data.head()

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,year,in_hospital,...,id_399.0,id_441.0,id_449.0,id_625.0,id_629.0,id_799.0,q_share,r_share,generic_share_q,generic_share_r
0,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,292190.95,2014,1,...,0,0,0,0,0,0,0.000085,0.000036,0.236031,0.053874
1,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,128832.00,2014,1,...,0,0,0,0,0,0,0.000038,0.000016,0.236031,0.053874
2,114.0,解熱鎮痛消炎剤,661140020.0,インドメタシン坐剤 ５０ｍｇ,1145700J3012,19.3,0.0,106328.00,2014,1,...,0,0,0,0,0,0,0.000031,0.000013,0.236031,0.053874
3,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,658257.00,2014,0,...,0,0,0,0,0,0,0.000192,0.000081,0.236031,0.053874
4,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,418727.50,2014,0,...,0,0,0,0,0,0,0.000122,0.000051,0.236031,0.053874


In [6]:
data_est=data.dropna(subset=['lag_mean_price'])
ids=data_est.iloc[:,83:115]
X=data_est[["year"]+list(ids.columns)]
y=data_est["lag_mean_price"]
# X = sm.add_constant(X)  # 定数項を追加
model = sm.OLS(y, X).fit()

# モデルの要約を表示
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:         lag_mean_price   R-squared (uncentered):                   0.934
Model:                            OLS   Adj. R-squared (uncentered):              0.934
Method:                 Least Squares   F-statistic:                          1.204e+04
Date:                Wed, 06 Sep 2023   Prob (F-statistic):                        0.00
Time:                        09:54:38   Log-Likelihood:                     -1.7555e+05
No. Observations:               28030   AIC:                                  3.512e+05
Df Residuals:                   27997   BIC:                                  3.514e+05
Df Model:                          33                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [7]:
missing=data[data['lag_mean_price'].isnull()]
missing=missing[["year"]+list(ids.columns)]
# missing["const"]=1
predicted_values=model.predict(missing)
predicted_values

0        27.034277
1        27.034911
2        27.034998
3        27.032856
4        27.033786
           ...    
1618    131.651214
1619    131.484806
1620    131.578123
1621    131.594177
1622    131.651462
Length: 1623, dtype: float64

In [8]:
missing["lag_mean_price"]=predicted_values

In [9]:
data.update(missing)

In [10]:
# data.columns
data.to_csv('../merged/len3_ndb_agg_blp_imputed_lags.csv',encoding="shift-jis")

In [11]:
data.shape

(29653, 117)